In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
import contractions
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob
import re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import ipywidgets
import swifter
import gc
import os
gc.collect()

In [ ]:
glasssdoor_data_dir = "/ifs/projects/amirgo-identification/glassdoor_data"
glassdoor_reviews = pd.read_csv(os.path.join(glassdoor_data_dir, "reviews_new_processed.csv"))
glassdoor_reviews = glassdoor_reviews.loc[glassdoor_reviews['is_current_job'] == 1,]
tokenizer = TweetTokenizer()
stop_words = STOP_WORDS
custom_stop_words = ['people', 'employee', 'employees', 'work', 'working', 'company', 'great', 'good', 'lot']
stop_words = set(list(stop_words) + custom_stop_words)

lemmatizer = WordNetLemmatizer()
# getting rid of these tokens that are not included in the pre-defined punctuation list
# no better lists existing
punctuation = string.punctuation + '–...…’“”'
# removing bullet points and numbers
re_number = r"[0-9]+(\.)?"
glassdoor_reviews = glassdoor_reviews.dropna(subset=['pros', 'cons']).astype({'pros':'str', 'cons':'str'})

In [ ]:
def clean_text(text):
    text = contractions.fix(text).lower()
    cleaned_toks = [lemmatizer.lemmatize(t) for t in tokenizer.tokenize(text.replace('.',' '))
                   if t not in punctuation and t not in stop_words and re.match(re_number, t) is None]
    return ' '.join(cleaned_toks)

In [ ]:
%%time
glassdoor_reviews['pros_cleaned'] = glassdoor_reviews.swifter.allow_dask_on_strings(enable=True).apply(lambda row : clean_text(row['pros']), axis=1)

In [ ]:
%%time
glassdoor_reviews['cons_cleaned'] = glassdoor_reviews.swifter.allow_dask_on_strings(enable=True).apply(lambda row : clean_text(row['cons']), axis=1)

In [ ]:
glassdoor_reviews = glassdoor_reviews.drop(columns=["pros", "cons", "feedback",
                                                   "pros_processed_train", "cons_processed_train", "pros_processed_eval", "cons_processed_eval"])
glassdoor_reviews.to_csv(os.path.join(glassdoor_data_dir, 'glassdoor_reviews_custom.csv'))

In [ ]:
glassdoor_reviews[["employer_id", "employer_name", "employer_type",
                   "num_employees", "revenue_annual", "industry", "sector"]].drop_duplicates().to_csv(
    os.path.join(glassdoor_data_dir, 'glassdoor_companies.csv'))

In [ ]:
glassdoor_reviews[["review_id", "employer_id", "gender", "len_employment", "pros_cleaned", "cons_cleaned"]].to_csv(
    os.path.join(glassdoor_data_dir, 'glassdoor_reviews_only.csv'))

In [ ]:
with open('/ifs/projects/amirgo-identification/BTM/glassdoor_data/pros.txt', 'w') as f:
    for item in glassdoor_reviews['pros_cleaned'].tolist():
        f.write("%s\n" % item)
with open('/ifs/projects/amirgo-identification/BTM/glassdoor_data/cons.txt', 'w') as f:
    for item in glassdoor_reviews['cons_cleaned'].tolist():
        f.write("%s\n" % item)

In [ ]:
# shows companies are matched with more than one cities - might just be city of reviewer
emps = glassdoor_reviews[['employer_id', 'metro_name']].drop_duplicates()
ids = emps['employer_id']
emps[ids.isin(ids[ids.duplicated()])].sort_values('employer_id')

In [ ]:
glassdoor_reviews['datetime']

In [ ]:
# shows companies are matched with more than one cities - might just be city of reviewer
emps = glassdoor_reviews[['employer_id', 'revenue_annual']].drop_duplicates()
ids = emps['employer_id']
emps[ids.isin(ids[ids.duplicated()])].sort_values('employer_id')